--- Day 17: Clumsy Crucible ---

The lava starts flowing rapidly once the Lava Production Facility is operational. As you leave, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.

As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!

You land near the gradually-filling pool of lava at the base of your new lavafall. Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large crucibles on wheels.

The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.

To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible from the lava pool to the machine parts factory. To do this, you need to minimize heat loss while choosing a route that doesn't require the crucible to go in a straight line for too long.

Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.

For example:

```
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
```

Each city block is marked by a single digit that represents the amount of heat loss if the crucible enters that block. The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)

Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move at most three blocks in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.

One way to minimize heat loss is this path:

```
2>>34^>>>1323
32v>>>35v5623
32552456v>>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>
```

This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only 102.

Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, what is the least heat loss it can incur?

Your puzzle answer was 791.

In [ ]:
import heapq

def read_input(filePath):
    with open(filePath, 'r') as file:
        return file.read()
    
def read_heatmap(text):
    return [[int(char) for char in line] for line in text.strip().splitlines()]

def print_heatmap(heatmap):
    for line in heatmap:
        print(''.join([f'{i: <4}' for i in line]))

def create_graph(heatmap, allowed_movements):
    #
    # Example heatmap (cost matrix)
    #
    #   | 0  1  2  3  4  5  6  7  8  9 10 11 12
    #   |---------------------------------------
    # 0 | 2  4  1  3  4  3  2  3  1  1  3  2  3  
    # 1 | 3  2  1  5  4  5  3  5  3  5  6  2  3
    # 2 | 3  2  5  5  2  4  5  6  5  4  2  5  4
    # 3 | 3  4  4  6  5  8  5  8  4  5  4  5  2
    #   |
    #
    # Graph is mapping allowed movements, example is for (0,0)
    #
    # lr: you can move left or right when moving out of node
    # ud: you can move up or down when moving out of node
    #   
    # {
    #   (0,0)_lr: { (1,0)_ud: 4, (2,0)_ud: 5, (3,0)_ud: 8 },
    #   (0,0)_ud: { (0,1)_lr: 3, (0,2)_lr: 6, (0,3)_lr: 9 },
    # }  
    #
    height = len(heatmap)
    width = len(heatmap[0])
    graph = {}
    for y in range(height):
        for x in range(width):
            graph[(x, y, 'lr')] = {}
            # positive movements along x-axis
            for dx in allowed_movements:
                if 0 <= x + dx < width:
                    graph[(x, y, 'lr')][(x + dx, y, 'ud')] = sum([heatmap[y][i] for i in range(x+1, x+dx+1)])
            # negative movements along x-axis
            for dx in allowed_movements:
                if 0 <= x - dx < width:
                    graph[(x, y, 'lr')][(x - dx, y, 'ud')] = sum([heatmap[y][i] for i in range(x - dx, x)])
            graph[(x, y, 'ud')] = {}
            # positive movements along y-axis
            for dy in allowed_movements:
                if 0 <= y + dy < height:
                    graph[(x, y, 'ud')][(x, y + dy, 'lr')] = sum([heatmap[i][x] for i in range(y + 1, y + dy + 1)])
            # negative movements along y-axis
            for dy in allowed_movements:
                if 0 <= y - dy < height:
                    graph[(x, y, 'ud')][(x, y - dy, 'lr')] = sum([heatmap[i][x] for i in range(y - dy, y)])
    return graph

def dijkstra(graph, start):
    #
    # Initialize
    # ~~~~~~~~~~
    # - Create distances dictionary for vertices with all distances set to infinity
    # - Set distance for start node to 0 in distances dictionary
    # - Priority queue holds an ordered set of unvisited nodes
    #
    distances = {vertex: (float('infinity'), []) for vertex in graph}
    distances[start] = (0, [start])
    priority_queue = [(0, start)]
    #
    # Loop
    # ~~~~
    # - Pop vertex from unvisited set with the smallest distance
    # - For each neighbor of current vertex
    #   - Calculate distance through current vertex
    #   - Update distance of neighbor if it is smaller than current distance
    #   - Add neighbor to unvisited set
    # - Once unvisited set is empty, algorithm is finished
    #
    while priority_queue:
        dist_to_vertex, vertex = heapq.heappop(priority_queue)
        path_to_vertex = distances[vertex][1]
        for neighbor, neighbor_delta in graph[vertex].items():
            dist_through_vertex = dist_to_vertex + neighbor_delta
            dist_to_neighbor = distances[neighbor][0]
            if dist_through_vertex < dist_to_neighbor:
                distances[neighbor] = (dist_through_vertex, [*path_to_vertex, neighbor])
                heapq.heappush(priority_queue, (dist_through_vertex, neighbor))
    return distances

input = read_input('sample.txt')
heatmap = read_heatmap(input)

graph = create_graph(heatmap, [1, 2, 3])
distances = dijkstra(graph, (0, 0, 'lr'))

for (key, value) in list(distances.items())[-2:]:
    print(key, value)

--- Part Two ---

The crucibles of lava simply aren't large enough to provide an adequate supply of lava to the machine parts factory. Instead, the Elves are going to upgrade to ultra crucibles.

Ultra crucibles are even more difficult to steer than normal crucibles. Not only do they have trouble going in a straight line, but they also have trouble turning!

Once an ultra crucible starts moving in a direction, it needs to move a minimum of four blocks in that direction before it can turn (or even before it can stop at the end). However, it will eventually start to get wobbly: an ultra crucible can move a maximum of ten consecutive blocks without turning.

In the above example, an ultra crucible could follow this path to minimize heat loss:

```
2>>>>>>>>1323
32154535v5623
32552456v4254
34465858v5452
45466578v>>>>
143859879845v
445787698776v
363787797965v
465496798688v
456467998645v
122468686556v
254654888773v
432267465553v
```

In the above example, an ultra crucible would incur the minimum possible heat loss of 94.

Here's another example:

```
111111111111
999999999991
999999999991
999999999991
999999999991
```

Sadly, an ultra crucible would need to take an unfortunate path like this one:

```
1>>>>>>>1111
9999999v9991
9999999v9991
9999999v9991
9999999v>>>>
```

This route causes the ultra crucible to incur the minimum possible heat loss of 71.

Directing the ultra crucible from the lava pool to the machine parts factory, what is the least heat loss it can incur?

Your puzzle answer was 900.

In [ ]:
input = read_input('sample.txt')
heatmap = read_heatmap(input)

graph = create_graph(heatmap, [4, 5, 6, 7, 8, 9, 10])
distances = dijkstra(graph, (0, 0, 'lr'))

for (key, value) in list(distances.items())[-2:]:
    print(key, value)